In [1]:
!pip install -U spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 58.3 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.8.5
    Uninstalling spacy-3.8.5:
      Successfully uninstalled spacy-3.8.5


In [2]:
!python -m spacy init fill-config /kaggle/input/ner-resume-input-final/base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [3]:
!python -m spacy train /kaggle/input/ner-resume-input-final/config.cfg \
--output /kaggle/working/output \
--paths.train /kaggle/input/ner-train-final/train_data.spacy \
--paths.dev /kaggle/input/ner-train-final/test_data.spacy \
--gpu-id 0

✔ Created output directory: /kaggle/working/output
ℹ Saving to output directory: /kaggle/working/output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2025-06-01 14:33:28,431] [INFO] Set up nlp object from config
[2025-06-01 14:33:28,451] [INFO] Pipeline: ['tok2vec', 'ner']
[2025-06-01 14:33:28,454] [INFO] Created vocabulary
[2025-06-01 14:33:29,414] [WARNING] [W112] The model specified to use for initial vectors (en_core_web_sm) has no vectors. This is almost certainly a mistake.
[2025-06-01 14:33:29,416] [INFO] Added vectors: en_core_web_sm
[2025-06-01 14:33:29,416] [INFO] Finished initializing nlp object
[2025-06-01 14:33:44,993] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ----------

**MODEL TESTING**

In [4]:
# Load model
import spacy
nlp_ner = spacy.load('/kaggle/input/output/pytorch/default/1/output/model-best')

/usr/local/lib/python3.11/dist-packages/spacy/util.py:922: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.4.4 and may not be 100% compatible with the current version (3.8.7). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [5]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 81.1 MB/s eta 0:00:00


In [6]:
import sys, fitz
fname = '/kaggle/input/testresume/Alice Clark CV.pdf'
doc = fitz.open(fname)
text = ""
for page in doc:
    text = text + str(page.get_text())

tx = " ".join(text.split('\n'))
print(tx)

Alice Clark  AI / Machine Learning    Delhi, India Email me on Indeed  •  20+ years of experience in data handling, design, and development  •  Data Warehouse: Data analysis, star/snow flake scema data modelling and design specific to  data warehousing and business intelligence  •  Database: Experience in database designing, scalability, back-up and recovery, writing and  optimizing SQL code and Stored Procedures, creating functions, views, triggers and indexes.  Cloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL Azure,  Stream Analytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure data lake  analytics(U-SQL)  Willing to relocate anywhere    WORK EXPERIENCE  Software Engineer  Microsoft – Bangalore, Karnataka  January 2000 to Present  1. Microsoft Rewards Live dashboards:  Description: - Microsoft rewards is loyalty program that rewards Users for browsing and shopping  online. Microsoft Rewards members can earn points when searching with Bing, bro

In [7]:
doc = nlp_ner(tx)
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}- {ent.text}')

NAME                          - Alice Clark
SKILLS                        - Data analysis
SKILLS                        - Microsoft Azure cloud services
SKILLS                        - SQL
SKILLS                        - Power BI
SKILLS                        - Web Job
SKILLS                        - Power BI
DESIGNATION                   - Software Engineer
INSTITUTE                     - Indian Institute of Technology – Mumbai
SKILLS                        - Machine Learning
SKILLS                        - Big Data Handling
SKILLS                        - problem solving
SKILLS                        - communication
SKILLS                        - maintains cordial relationship with project


In [8]:
from spacy import displacy
displacy.render(doc, style="ent", jupyter=True)

**RANKING RESUMES**

In [9]:
import spacy
import os
import glob
import numpy as np
import pandas as pd

In [10]:
def extract_entities(text, target_labels=None):
    doc = nlp(text)
    if target_labels:
        entities = [ent.text for ent in doc.ents if ent.label_ in target_labels]
    else:
        entities = [ent.text for ent in doc.ents]
    return entities

In [11]:
def entity_similarity(jd_entities, resume_entities):
    """
    For each entity from the job description, compute the maximum similarity score
    to any entity in the resume. Return the average of these best match scores.
    """
    scores = []
    for jd_ent in jd_entities:
        jd_ent_doc = nlp(jd_ent)
        best_sim = 0.0
        for res_ent in resume_entities:
            res_ent_doc = nlp(res_ent)
            sim = jd_ent_doc.similarity(res_ent_doc)
            best_sim = max(best_sim, sim)
        scores.append(best_sim)
    if scores:
        return np.mean(scores)
    return 0.0

In [12]:
def text_similarity(jd_text, resume_text):
    """
    Use spaCy’s document-level similarity. This works well if your spaCy model
    uses vector representations that capture semantic meaning.
    """
    doc_jd = nlp(jd_text)
    doc_resume = nlp(resume_text)
    return doc_jd.similarity(doc_resume)

In [13]:
def resume_score(jd_text, resume_text, w_entity=0.5, w_text=0.5, target_labels=None):
    """
    Compute a combined matching score between the job description and a resume.
    
    Parameters:
      jd_text (str): The job description text.
      resume_text (str): The candidate's resume text.
      w_entity (float): Weight given to the entity-level matching score.
      w_text (float): Weight given to the overall text similarity score.
      target_labels (list or None): If provided, only these entity types are considered 
                                    (e.g., ["SKILL", "EDUCATION"]). If None, all entities are used.
    
    Returns:
      combined_score (float): A weighted score reflecting the similarity between JD and resume.
    """
    jd_entities = extract_entities(jd_text, target_labels)
    resume_entities = extract_entities(resume_text, target_labels)
    
    # Compute entity-based score (ranges from 0 to 1)
    ent_score = entity_similarity(jd_entities, resume_entities)
    # Compute full text similarity score (ranges from 0 to 1 typically)
    txt_score = text_similarity(jd_text, resume_text)
    
    # Combined score using a weighted average
    combined_score = w_entity * ent_score + w_text * txt_score
    return combined_score

In [14]:
from pypdf import PdfReader
nlp = spacy.load("en_core_web_sm")
def extract_pdf_text(pdf_path):
    """
    Extract and concatenate text from all pages of a PDF file.
    """
    text = ""
    try:
        reader = PdfReader(pdf_path)
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + " "
    except Exception as e:
        print(f"Error extracting text from {pdf_path}: {e}")
    return text.strip()

In [15]:
import re
def extract_candidate_details(text):
    """
    Use spaCy NER to extract candidate name and email.
    For the name, use the first detected 'PERSON' entity.
    For the email, look for an entity with label 'EMAIL' or use a regex fallback.
    """
    doc = nlp(text)
    
    name_entities = [ent.text.strip() for ent in doc.ents if ent.label_ == "PERSON"]
    name = name_entities[0] if name_entities else "N/A"
    
    email_entities = [ent.text.strip() for ent in doc.ents if ent.label_ == "EMAIL"]
    if email_entities:
        email = email_entities[0]
    else:
        # Fallback to regex to extract email.
        email_matches = re.findall(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}", text)
        email = email_matches[0] if email_matches else "N/A"
        
    return name, email
     

In [16]:
def load_job_description(file_path):
    """
    Read and return the job description text from a file.
    """
    with open(file_path, "r", encoding="utf8") as f:
        return f.read()

In [17]:
jd_text = load_job_description("/kaggle/input/job-description/jd_web_developer.txt")
target_labels = ["SKILL", "EXPERIENCE", "QUALIFICATION"]
resume_folder = "/kaggle/input/testresume"
resume_files = glob.glob(os.path.join(resume_folder, "*.pdf"))
data = []
for resume_path in resume_files:
    resume_text = extract_pdf_text(resume_path)
    if not resume_text:
        print(f"Could not extract text from {resume_path}")
        continue
        
        # Extract candidate details.
    name, email = extract_candidate_details(resume_text)
        # Compute similarity score between the job description and the resume.
    similarity_score = resume_score(jd_text, resume_text, w_entity=0.5, w_text=0.5, target_labels=target_labels)
        
        # Append details to the data list.
    data.append({
        "Name": name,
        "Email": email,
        "Similarity": similarity_score
        })

/tmp/ipykernel_19/2807786875.py:8: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  return doc_jd.similarity(doc_resume)


In [18]:
df = pd.DataFrame(data)
df.sort_values(by="Similarity", ascending=False, inplace=True)
df[['Name','Email', 'Similarity']]

,Name,Email,Similarity
0,Huỳnh Chí Trung\nBACKEND,huynhchitrung020503@gmail.com,0.413349
1,Alice Clark \nAI / Machine Learning,N/A,0.393511
